In [9]:
# This verion can be used to check if the project operates correctly or not without the use of MySQL
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
print("Imporing libraries done")

window = tk.Tk()
window.title("Face Recognition system")

T = tk.Label(window, text="Facial Recognition System", font=("Slab serif",20))
T.grid(column=1, row=0)

Name = tk.Label(window, text="Name", font=("Slab serif",20))
Name.grid(column=0, row=1)
t1 = tk.Entry(window, width=50, bd=3)
t1.grid(column=1, row=1)
 
Age = tk.Label(window, text="Age", font=("Slab serif",20))
Age.grid(column=0, row=2)
t2 = tk.Entry(window, width=50, bd=3)
t2.grid(column=1, row=2)
 
Address = tk.Label(window, text="Address", font=("Slab serif",20))
Address.grid(column=0, row=3)
t3 = tk.Entry(window, width=50, bd=3)
t3.grid(column=1, row=3)

# After installing openCV using the pip install opencv-python command, we'll import the library
def generate_dataset(): # Creating a function called generate dataset, since this is what will take user's face as input whenever we add a new user
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result', 'Please provide all details of the user')
    else:
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml") #Input the cascade classifier which includes all what I was talking about in terms of how to detect face in general, haar-like feature, integral image, adaboost, and classifier cascades
        def face_cropped(img): # This function is to help the computer read the user's face and give us back its coordinates
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # Conver Image from RGB colors to grey scale (from 0 to 255) with 0 being black and 255 being white and the number in between being grey
            faces = face_classifier.detectMultiScale(gray, 1.2, 5) # Create a variable called faces to give us back 4 values (coordinates) from the function called detect multiscale. These values are the coordinates of the place where the person's face is located.
            # scaling factor = 1.3, not sure I quite understand it but I think it's the size of the recantgle?! Parameter specifying how much the image size is reduced at each image scale. By rescaling the input image, you can resize a larger face to a smaller one, making it detectable by the algorithm. i.e. reduce the size by 5%, you increase the chance of a matching size with the model for detection is found.
            # minimum neighbor = 5, I think it's how many weak classification or recatangles around the main one. Parameter specifying how many neighbours each candidate rectangle should have to retain it. This parameter will affect the quality of the detected faces. Higher value results in fewer detections but with higher quality. 3~6 is a good value for it.
            if faces is (): # if faces is empty or did not read anything, return nothing
                return None 
            for (x,y,w,h) in faces: 
                cropped_face = img[y:y+h,x:x+w] # X is the pixel of the x-axis on the screen and x+w is x-coordinate + width, giving us the 4 dimensions of the rectangle
            return cropped_face
    
           
        cap = cv2.VideoCapture(0) # 0 means access camera from laptop/webcam I think if 1 or -1 then it's external camera
        id =10 # first authorized person, if second person give id=2
        img_id = 0 # number of images for first authorized person
        
        while True: # After opening the camera, create a while loop that keeps taking pictures as long as face_cropped is true
            ret, frame = cap.read() # frame refers to image i believe
            if face_cropped(frame) is not None: # if the face_cropped function is acheived, we will make frame = to img. Make frame enter the face_cropped function and turn it into greyscale...etc.
                img_id+=1 # save images with the following index user.1.img_id
                face = cv2.resize(face_cropped(frame), (200,200)) #resize frame and store it into a variable called face. We resize because we want the downloaded picture to show a large view of your face
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) # turn face into grey scale
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                # (50,50) is the origin point from where text is going to be writte
                # font scale = 1
                # thinckness = 2
                # font name is hershey complex
                 
                cv2.imshow("Cropped face", face)
                 
            if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter key, so that when we press enter it stops or we reached 200 images 
                break
        cap.release() # release camera not sure what it means
        cv2.destroyAllWindows()
        messagebox.showinfo('Result','Generating Dataset Completed')

 
def train_classifier(): # Create a function that takes data directory (my collected data) located at directory of "data" file I created
    data_dir = "C:/Users/Adham/Desktop/Face Data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)] # Make the path be data_dir path + the data_dir of the image after listing it and storing it in f
    # for example, C:\Users\Adham\Desktop\Face recognizer\data\user.1.1
    # for example, C:\Users\Adham\Desktop\Face recognizer\data\user.1.2...etc.
     
    faces = [] # create empty list
    ids = [] 
     
    for image in path:
        img = Image.open(image).convert('L') # Change image to gray image
        imageNp = np.array(img, 'uint8') # Change image to array format so we can split it
        id = int(os.path.split(image)[1].split(".")[1])
        # C:\Users\Adham\Desktop\Face recognizer\data all of this is stored in array with idex [0]
        # C:\Users\Adham\Desktop\Face recognizer\data\[0] user.1.1 this is index [1]
        # After splitting index [1] by "." We'll have 3 index user[0], .1 [1], .1[2]
        # Then, after splitting, we'll store the correspoding value of splitted index [1] in id value since it's the person's id
         
        faces.append(imageNp)
        ids.append(id) # add id to the array called ids
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    #clf = cv2.face.LBPHFaceRecognizer_create()
    #clf = cv2.createLBPHFaceRecognizer()


    clf.train(faces,ids) # train data 
    clf.write("classifier.xml") # save data
    messagebox.showinfo('Result','Training Dataset Completed')

def detect_face(): 
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf): #Create function to form a rectangle around the face
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Turn image into grey scale image
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors) # detectMultiScale() method is commonly used from libraries 
        # like OpenCV to identify objects (such as faces) within an image. It utilizes the Haar-like features, integral imaging, creating classifier cascades  
        # which detects objects at multiple scales in an image.
         
        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 ) # Create a rectangle around face with coordinates taken from features with color and thickness 2
            # Takes into consideration real image not grey scale 
            id, pred = clf.predict(gray_img[y:y+h,x:x+w]) # Predict from grey scale image 
            # It is very important to note that this id is the one predicted by the classifier, not the one we put in the beginning
            confidence = int(100*(1-pred/300)) # A formula to know whether my user is authorized or not according to my confidence value
            
            
            if confidence>65:
                if id==1: # First person in database 
                    cv2.putText(img, "Adham", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA) # Put text and all information on real image
                if id==2: # First person in database 
                    cv2.putText(img, "Adham Mask", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                if id==3: # First person in database 
                    cv2.putText(img, "Mansy", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                if id==4: # First person in database 
                    cv2.putText(img, "Eng. Kareem", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA) # Put text and all information on real image
                if id==5: # First person in database 
                    cv2.putText(img, "Mashad", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA) # Put text and all information on real image
                if id==6: # First person in database 
                    cv2.putText(img, "Ahmed Khaled", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA) # Put text and all information on real image
            else:
                cv2.putText(img, "Unknown Person", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
         
        return img
     
    # loading classifier
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
     
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
     
    video_capture = cv2.VideoCapture(0)
     
    while True:
        ret, img = video_capture.read()
        img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
        cv2.imshow("face Detection", img)
         
        if cv2.waitKey(1)==13: 
            break
    video_capture.release()
    cv2.destroyAllWindows()

b1 = tk.Button(window, text="Generate Dataset", font=("Slab serif",20), bg="pink", fg="black", command = generate_dataset)
b1.grid(column=0, row=5)

b2 = tk.Button(window, text="Training the Dataset", font=("Slab serif",20),bg="orange",fg="black", command = train_classifier )
b2.grid(column=1, row=5)
 
b3 = tk.Button(window, text="Detect Face", font=("Slab serif",20), bg="red", fg="black", command = detect_face)
b3.grid(column=3, row=5)
 
def message_box():
    messagebox.showinfo("Info page", "This is a Facial Recognition System able to detect new faces real-time made by:\nAdham Hossam - E-mail: A.Hossammahmoud2164@nu.edu.eg \nAmro Hossam \nShahd Hamdy \nAdham Magdy")

b4 = tk.Button(window, text="Contact us", font=("Slab serif",13), bg="pink", fg="black", command = message_box)
b4.place(x=0, y=270)

window.geometry("900x300")
window.mainloop()

<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Adham\AppData\Local\Temp\ipykernel_21884\1307448787.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is (): # if faces is empty or did not read anything, return nothing


Imporing libraries done
